# Crop YOLOv4 people

## Path definitions - change to your own

In [16]:
import pandas as pd
import cv2
import os
from matplotlib import pyplot as plt

import ipywidgets as widgets
from PIL import Image

In [3]:
IMAGES_PATH = "/content/drive/MyDrive/TFM/OriginalInput"
ANNOTATIONS = "/content/drive/MyDrive/TFM/OriginalInput/annotateTest.csv"
YOLOV4_RESULTS_CSV = "/content/drive/MyDrive/TFM/yolov4_results/transform_90.csv"
OUT_IMAGES = "/content/drive/MyDrive/TFM/CROP3"

ANNOTATION_COLUMNS = ["filename", "x_min", "y_min", "x_max", "y_max", "class"]
NEW_ANNOTATIONS = pd.DataFrame(columns=ANNOTATION_COLUMNS)

MIN_AREA = (64, 64)

Here Google Drive storage is loaded. You can omit this cell if you are not using it.

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%mkdir {OUT_IMAGES}

In [11]:
df = pd.read_csv(YOLOV4_RESULTS_CSV)
df

,filename,x,y,x2,y2,class
0,/content/drive/MyDrive/TFM/OriginalInput/Ayaga...,618,87,828,694,person
1,/content/drive/MyDrive/TFM/OriginalInput/Ayaga...,1503,315,1581,444,person
2,/content/drive/MyDrive/TFM/OriginalInput/Ayaga...,1785,319,1909,507,person
3,/content/drive/MyDrive/TFM/OriginalInput/Ayaga...,1716,317,1828,484,person
4,/content/drive/MyDrive/TFM/OriginalInput/Ayaga...,1567,334,1648,449,person
...,...,...,...,...,...,...
1493,/content/drive/MyDrive/TFM/OriginalInput/Teror...,935,435,989,525,person
1494,/content/drive/MyDrive/TFM/OriginalInput/Teror...,1147,398,1336,950,person
1495,/content/drive/MyDrive/TFM/OriginalInput/Teror...,904,448,1006,533,person
1496,/content/drive/MyDrive/TFM/OriginalInput/Teror...,1758,462,1854,575,person


In [ ]:
def crop_image(index, object_detection, save_path, original_annotations=None, min_area=64*64):
  global NEW_ANNOTATIONS

  img = cv2.imread(object_detection['filename'])
  height, width = img.shape[:2]

  if width < min_area[1] or height < min_area[0]:
    return False, None

  crop_img = img[object_detection['y']:object_detection['y2'], object_detection['x']:object_detection['x2']]
  name = f'{index}_{os.path.basename(object_detection["filename"])}'
  cv2.imwrite(os.path.join(save_path, name), crop_img)

  if NEW_ANNOTATIONS is not None and original_annotations is not None:
    df_ann = pd.read_csv(original_annotations, header=None)
    df_ann.columns = ["filename", "x_min", "y_min", "x_max", "y_max", "class"]
    df_ann = df_ann[df_ann['filename'].str.endswith(os.path.basename(object_detection['filename']))]

    for index, row in df_ann.iterrows():
      x_min = row['x_min'] - object_detection['x']
      y_min = row['y_min'] - object_detection['y']
      x_max = (object_detection['x2'] - object_detection['x']) - (object_detection['x2'] -  row['x_max'])
      y_max = (object_detection['y2'] - object_detection['y']) - (object_detection['y2'] -  row['y_max'])

      if (x_min >= 0 and y_min >= 0 and x_max >= 0 and y_max >= 0) and (x_min <= (object_detection['x2'] - object_detection['x']) and y_min <= (object_detection['y2'] - object_detection['y']) and x_max <= (object_detection['x2'] - object_detection['x']) and y_max <= (object_detection['y2'] - object_detection['y'])):
        NEW_ANNOTATIONS = NEW_ANNOTATIONS.append({
            'filename': os.path.join(save_path, name),
            'x_min': x_min,
            'y_min': y_min,
            'x_max': x_max,
            'y_max': y_max,
            'class': row['class']
        }, ignore_index=True)
        print(NEW_ANNOTATIONS.tail().iloc[:, 1:5])
      else:
        print('Not valid annotation placing')

    return True, NEW_ANNOTATIONS
  return False, None

In [38]:
def crop_image_demo(filename):
  crop_imgs = []
  object_detections = df[df['filename'].str.endswith(filename)]
  img = cv2.imread(filename)
  for index, row in object_detections.iterrows():
    crop_imgs.append(cv2.cvtColor(img[row['y']:row['y2'], row['x']:row['x2']], cv2.COLOR_BGR2RGB))
  return crop_imgs

## Crop and store into 'OUT_IMAGES'

In [ ]:
for index, row in df.iterrows():
  done, annotation = crop_image(index, row, OUT_IMAGES, ANNOTATIONS, MIN_AREA)
  file_name = f'{index}_{os.path.basename(row["filename"])}'

  if done:
    print(f'[{index}] {os.path.join(OUT_IMAGES, file_name)}')
    NEW_ANNOTATIONS = annotation
  else:
    print(f'[{index}] {row["filename"]} ignored because has less area than {MIN_AREA}px')

if NEW_ANNOTATIONS is not None:
  NEW_ANNOTATIONS.to_csv(os.path.join(OUT_IMAGES, 'annotateTest.csv'), header=None, index=False)

## Crop demo

In [9]:
image_list_siani = os.listdir(IMAGES_PATH)
image_picker_siani = widgets.Dropdown(options=image_list_siani, value=image_list_siani[0], disabled=False)
image_picker_siani

Dropdown(options=('Ayagaures_frame_18_45_40_000.jpg', 'Ayagaures_frame_18_15_22_000.jpg', 'Arucas_frame_01_28_…

In [ ]:
crops = crop_image_demo(os.path.join(IMAGES_PATH, image_picker_siani.value))
img = Image.open(os.path.join(IMAGES_PATH, image_picker_siani.value))
i = 1

fig = plt.figure(figsize=(10, 20))
fig.add_subplot(3,1,1)
plt.imshow(img)
plt.axis('off')

for crop in crops:
  fig.add_subplot(1,len(crops),i)
  plt.imshow(Image.fromarray(crop))
  plt.axis('off')
  i += 1